In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/train-images-idx3-ubyte


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        # Third convolutional layer
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        # Fourth convolutional layer
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        
        # MaxPool layer after each convolutional layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 1 * 1, 512)   # Adjusted for the new dimensions after pooling
        self.fc2 = nn.Linear(512, 256)           # Additional fully connected layer
        self.fc3 = nn.Linear(256, 128)           # Additional fully connected layer
        self.fc4 = nn.Linear(128, 64)            # Additional fully connected layer
        self.fc5 = nn.Linear(64, 10)             # Assuming 10 output classes

    def forward(self, x):
        # Apply first convolutional layer, ReLU, then MaxPool
        x = self.pool(F.relu(self.conv1(x)))  
        
        # Apply second convolutional layer, ReLU, then MaxPool
        x = self.pool(F.relu(self.conv2(x)))  
        
        # Apply third convolutional layer, ReLU, then MaxPool
        x = self.pool(F.relu(self.conv3(x)))  
        
        # Apply fourth convolutional layer, ReLU, then MaxPool
        x = self.pool(F.relu(self.conv4(x)))  
        
        # Flatten the output for the fully connected layers
        x = x.view(-1, 256 * 1 * 1)  # Adjusted size after pooling
        
        # Apply fully connected layers with ReLU activations
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        # Final output layer
        x = self.fc5(x)
        
        return x


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Define your transforms
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Assuming grayscale, adjust as needed
])

class CustomImageDataset(Dataset):
    def __init__(self, data, transform=None):
        # The data parameter here is a pandas DataFrame that already contains a split dataset
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract the label (first column) and pixel values (remaining columns)
        label = int(self.data.iloc[idx, 0])  # Convert label to integer
        image = self.data.iloc[idx, 1:].values.astype(np.float32)  # Pixels as float32
        image = image.reshape(28, 28)  # Reshape to 28x28 if images are 28x28 pixels

        # Convert the image to a PIL Image for the transformation
        image = Image.fromarray(image)  # Convert to PIL Image (default mode is 'L' for grayscale)

        # Apply transform (e.g., ToTensor)
        if self.transform:
            image = self.transform(image)

        # Convert label to tensor
        label = torch.tensor(label)

        return image, label


In [4]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm  # Import tqdm for progress bar
import numpy as np
from PIL import Image


# Load the CSV data
csv_file_path = '/kaggle/input/fashionmnist/fashion-mnist_train.csv'
data = pd.read_csv(csv_file_path)

# Split the data into train and validation (80% train, 20% validation)
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Assuming you already have the test set, just load it
test_data = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_test.csv')

# Create datasets for train, validation, and test with transform applied
train_dataset = CustomImageDataset(data=train_data, transform=transform)
val_dataset = CustomImageDataset(data=val_data, transform=transform)
test_dataset = CustomImageDataset(data=test_data, transform=transform)

# Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Assuming your dataset is split into train_loader and val_loader
# Define your model, loss function, and optimizer
model = SimpleCNN()  # Replace with your actual model
criterion = torch.nn.CrossEntropyLoss()  # Common loss for classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Early stopping parameters
early_stopping_patience = 5  # Number of epochs to wait for improvement in validation loss
best_val_loss = np.inf  # Start with a very large number
best_val_accuracy = 0.0  # Start with a low validation accuracy
epochs_without_improvement = 0  # Counter for epochs without improvement

# Path to save the best model
best_model_path = 'best_model.pth'

# Training Loop
num_epochs = 100  # Set number of epochs
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Wrap the training loop with tqdm for progress bar
    with tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}") as pbar:
        for inputs, labels in pbar:  # train_loader is your training DataLoader
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Clear gradients

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # Calculate loss

            # Backward pass and optimization
            loss.backward()  # Compute gradients
            optimizer.step()  # Update model parameters

            running_loss += loss.item()  # Accumulate loss for reporting

            # For accuracy calculation
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm description with current loss and accuracy
            pbar.set_postfix(loss=running_loss / (pbar.n + 1), accuracy=100 * correct / total)

    # Print the training loss and accuracy for this epoch
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

    # Validation after each epoch
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    # Wrap the validation loop with tqdm for progress bar
    with tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{num_epochs}") as pbar:
        with torch.no_grad():  # No gradients needed during evaluation
            for inputs, labels in pbar:  # val_loader is your validation DataLoader
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                # For accuracy calculation
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Update tqdm description with current validation loss and accuracy
                pbar.set_postfix(loss=val_loss / (pbar.n + 1), accuracy=100 * correct / total)

    # Print the validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    scheduler.step()  # Adjust the learning rate based on the scheduler

    # Optionally, print the current learning rate
    print(f"Current learning rate: {scheduler.get_last_lr()[0]:.6f}")
    
    # Early stopping check
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy  # Update best validation accuracy
        torch.save(model.state_dict(), best_model_path)  # Save the model with the best accuracy
        epochs_without_improvement = 0  # Reset counter
    else:
        epochs_without_improvement += 1  # Increment counter

    # If no improvement for 'early_stopping_patience' epochs, stop training
    if epochs_without_improvement >= early_stopping_patience:
        print("Early stopping triggered: No improvement in validation accuracy for 5 epochs.")
        break

Training Epoch 1/100: 100%|██████████| 1500/1500 [00:38<00:00, 39.41it/s, accuracy=78.9, loss=0.576]


Epoch [1/100], Loss: 0.5751, Accuracy: 78.86%


Validating Epoch 1/100: 100%|██████████| 375/375 [00:07<00:00, 47.03it/s, accuracy=85.2, loss=0.42]


Validation Loss: 0.4151, Validation Accuracy: 85.25%
Current learning rate: 0.001000


Training Epoch 2/100: 100%|██████████| 1500/1500 [00:37<00:00, 40.09it/s, accuracy=85.9, loss=0.397]


...

Validation Loss: 0.2467, Validation Accuracy: 92.43%
Current learning rate: 0.000031


Training Epoch 28/100: 100%|██████████| 1500/1500 [00:37<00:00, 39.65it/s, accuracy=95.9, loss=0.112]


Epoch [28/100], Loss: 0.1114, Accuracy: 95.88%


Validating Epoch 28/100: 100%|██████████| 375/375 [00:07<00:00, 47.12it/s, accuracy=92.5, loss=0.251]

Validation Loss: 0.2492, Validation Accuracy: 92.50%
Current learning rate: 0.000031
Early stopping triggered: No improvement in validation accuracy for 5 epochs.


In [5]:
# Load the best model for testing with weights_only=True
best_model = SimpleCNN()  # Initialize the model
best_model.load_state_dict(torch.load(best_model_path, weights_only=True))  # Load only the model weights
best_model.to(device)  # Move to the correct device


# Testing the best model
best_model.eval()  # Set model to evaluation mode
test_loss = 0.0
correct = 0
total = 0

# Wrap the testing loop with tqdm for progress bar
with tqdm(test_loader, desc="Testing") as pbar:
    with torch.no_grad():  # No gradients needed during evaluation
        for inputs, labels in pbar:  # test_loader is your test DataLoader
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = best_model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()

            # For accuracy calculation
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm description with current test loss and accuracy
            pbar.set_postfix(loss=test_loss / (pbar.n + 1), accuracy=100 * correct / total)

# Print the test results
test_loss /= len(test_loader)
test_accuracy = 100 * correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Testing: 100%|██████████| 313/313 [00:06<00:00, 49.46it/s, accuracy=92.8, loss=0.228]

Test Loss: 0.2261, Test Accuracy: 92.83%
